# Requisitos Previos

In [ ]:
#!pip install Flask-CORS
#!pip install neattext
#!pip install emoji
#!pip install scikit-learn==1.4.2 
#!pip install selenium

#!pip install joblib
#!pip install nltk
#!pip install Flask

In [1]:
import platform
import joblib
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...


True

In [2]:
from flask import request, jsonify, Flask

In [3]:
# Funcion axuiliar para saber si estoy en el collab y usar su path o el del proyecto de github
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False
    
from enum import Enum
# Funcion auxiliar para luego entrenar varios modelos con una sola ejecucion
class Modelos(Enum):
    LOGISTIC_REGRESSION = 'logistic_regression'
    DECISION_TREE = 'decision_tree'
    MULTINOMIAL = 'multinomial'
    BERNOULLI = 'bernoulli'
    GAUSIAN = 'gausian'

# Funcion procesador texto

In [4]:
import neattext.functions as nfx
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import emoji

wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'


stopwords_en = stopwords.words('english')
stopwords_en = set(stopwords_en).union(set(punctuation))

my_custom_stopwords = {'’', "n't", "'m", "'s", "'ve", '...', 'ca', "''", '``', '\u200d', 'im', 'na', "'ll", '..', 'u', "'re", "'d", '--', '”', '“', '\u200f\u200f\u200e', '....', 'ㅤ','\u200e\u200f\u200f\u200e', 'x200b', 'ive', '.-', '\u200e', '‘'}

stopwords_en = stopwords_en.union(my_custom_stopwords)


def preprocessing_function(text):
    words = []

    for word, tag in pos_tag(word_tokenize(nfx.clean_text(text))):
        word_lemmatized = wnl.lemmatize(word.lower(), pos=penn2morphy(tag))

        if '\u200b' in word_lemmatized:
            continue

        if word_lemmatized not in stopwords_en and not word_lemmatized.isdigit() and not emoji.purely_emoji(word_lemmatized):
            words.append(word_lemmatized)

    return words

# Carga modelo entrenado SUICIDIO

In [5]:
# MODIFICAR ESTOS PARAMETROS PARA LA CARGA
# --------------------------------------------------------
nombre_modelo_prev_entrenado = Modelos.LOGISTIC_REGRESSION.value
# usar formato '25k' para 25.000 filas ejemplo
cant_prev_entrenada = '50k'

path_base_modelo_generado = '/content/' if is_running_on_colab() else ('../tentativa_suicidio/entrenados/' if platform.system() == 'Linux' else '..\\tentativa_suicidio\\entrenados\\')
path_modelo_generado = path_base_modelo_generado + nombre_modelo_prev_entrenado + '_' + cant_prev_entrenada
# --------------------------------------------------------

suicide_model = joblib.load(path_modelo_generado + '_model.pkl')
suicide_vect = joblib.load(path_modelo_generado + '_vector.pkl')

print(type(suicide_model))
print(type(suicide_vect))

<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.feature_extraction.text.CountVectorizer'>


# Carga modelo entrenado VIOLENCIA

In [6]:
# MODIFICAR ESTOS PARAMETROS PARA LA CARGA
# --------------------------------------------------------
nombre_modelo_prev_entrenado = Modelos.LOGISTIC_REGRESSION.value
# usar formato '25k' para 25.000 filas ejemplo
cant_prev_entrenada = '50k'

path_base_modelo_generado = '/content/' if is_running_on_colab() else ('../violencia/entrenados/' if platform.system() == 'Linux' else '..\\violencia\\entrenados\\')
path_modelo_generado = path_base_modelo_generado + nombre_modelo_prev_entrenado + '_' + cant_prev_entrenada
# --------------------------------------------------------

violence_model = joblib.load(path_modelo_generado + '_model.pkl')
violence_vect = joblib.load(path_modelo_generado + '_vector.pkl')

print(type(violence_model))
print(type(violence_vect))

<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.feature_extraction.text.CountVectorizer'>


# Funcion predict SUICIDIO

In [7]:
# translator = Translator()

def get_tentativa_suicidio(text_input, english_text=False):
    texto_a_analizar = text_input #if english_text else translator.translate(text_input, dest='en').text
        
    texto_preprocesado = ' '.join(preprocessing_function(texto_a_analizar))
    texto_vectorizado = suicide_vect.transform([texto_preprocesado])

    return bool(suicide_model.predict(texto_vectorizado)[0])

# Funcion predict VIOLENCIA

In [8]:
# translator = Translator()

def get_texto_con_violencia(text_input, english_text=False):
    texto_a_analizar = text_input #if english_text else translator.translate(text_input, dest='en').text
        
    texto_preprocesado = ' '.join(preprocessing_function(texto_a_analizar))
    texto_vectorizado = violence_vect.transform([texto_preprocesado])

    return bool(violence_model.predict(texto_vectorizado)[0])

# Funcion obtener texto de url

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re

def get_texto(url):
    options = webdriver.FirefoxOptions()
    options.add_argument('--headless')
    driver = webdriver.Firefox(options=options)
    driver.get(url)

    texto = ""
    hay_read_more = False
    time.sleep(1)

    try:
        buttons = driver.find_elements(By.XPATH, "//button[contains(@id, '-read-more-button')]") 
        for button in buttons:
            if re.search(r"-read-more-button$", button.get_attribute("id")):
                if button.is_displayed():
                    hay_read_more = True
                    button.click()
                    break
        if(hay_read_more):
            button_id = button.get_attribute("id")
            div_id_pattern = re.sub(r"-read-more-button$", "-post-rtjson-content", button_id)
            texto_element = driver.find_element(By.XPATH, f"//div[contains(@id, '{div_id_pattern}')]")
        else:
            parent_div = driver.find_element(By.XPATH, "//div[@class='text-neutral-content']")
            texto_element = parent_div.find_element(By.XPATH, ".//div[contains(@id, '-post-rtjson-content')]")
            
        texto = texto_element.text
    finally:
        driver.close()


    return texto

# Funcion para traduccion

In [12]:
from deep_translator import GoogleTranslator

def translatedSentence(text):
    return GoogleTranslator(source='auto', target='en').translate(text=text)

# API

In [13]:
from flask_cors import CORS

app = Flask(__name__)

CORS(app)

@app.route("/")
def home():
    return "<p>API para predecir tentativas de suicidio en textos.</p>"

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Funciona"})


@app.route('/text_prediction', methods=['GET'])
def procesar_texto_get():
    texto = request.args.get('text')
    if texto:
        print(f"Texto recibido: '{texto}'")
        texto = texto.strip()
        if(texto.isspace() or texto == ""):
            return jsonify({"error": "No se pudo procesar el texto ya que esta vacío"})
        
        prediction_suicidio = get_tentativa_suicidio(texto)
        prediction_violencia = get_texto_con_violencia(texto)

        result = {"prediction_suicidio": prediction_suicidio, "prediction_violencia": prediction_violencia}

        print(f"Resultado: '{result}'")
        return jsonify(result)
    else:
        return jsonify({"error": "No se proporció un texto en la solicitud"})
    
@app.route("/text_prediction", methods=["POST"])
def procesar_texto_post():
    data = request.json
    texto = data.get("texto")
    if texto:
        print(f"Texto recibido: '{texto}'")
        texto = texto.strip()
        if(texto.isspace() or texto == ""):
            return jsonify({"error": "No se pudo procesar el texto ya que esta vacío"})
        
        texto_eng=translatedSentence(texto)
        prediction_suicidio = get_tentativa_suicidio(texto_eng)
        prediction_violencia = get_texto_con_violencia(texto_eng)

        result = {"prediction_suicidio": prediction_suicidio, "prediction_violencia": prediction_violencia}

        print(f"Resultado: '{result}'")
        return jsonify(result)
    else:
        return jsonify({"error": "No se proporcionó un texto en la solicitud"})
    
@app.route("/url_prediction", methods=["POST"])
def procesar_url_post():
    data = request.json
    url = data.get("url")
    if url:
        print(f"URL Recibida: '{url}'")
        texto = get_texto(url)
        print(f"Texto obtenido: '{texto}'")
        
        texto_eng=translatedSentence(texto)
        prediction_suicidio = get_tentativa_suicidio(texto_eng)
        prediction_violencia = get_texto_con_violencia(texto_eng)

        result = {"prediction_suicidio": prediction_suicidio, "prediction_violencia": prediction_violencia}

        print(f"Resultado: '{result}'")
        return jsonify(result)
    else:
        return jsonify({"error": "No se proporcionó la URL en la solicitud"})

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [23/May/2024 19:20:53] "OPTIONS /text_prediction HTTP/1.1" 200 -


Texto recibido: 'hola'


127.0.0.1 - - [23/May/2024 19:20:54] "POST /text_prediction HTTP/1.1" 200 -


Resultado: '{'prediction_suicidio': False, 'prediction_violencia': False}'


...

Resultado: '{'prediction_suicidio': True, 'prediction_violencia': True}'


127.0.0.1 - - [23/May/2024 19:29:16] "OPTIONS /text_prediction HTTP/1.1" 200 -


Texto recibido: 'él siempre le decía verdades absolutas directamente y ella, qué puta, simplemente comenzó a mirar su teléfono con esa cara de puta muerta y nunca se rió. No estoy seguro de por qué perdió el tiempo, no la va a reacondicionar de los chismes de la sociedad, mentiras en quince. minutos, alguna vez mujeres, somos tan estúpidos como ser honestos'


127.0.0.1 - - [23/May/2024 19:29:18] "POST /text_prediction HTTP/1.1" 200 -


Resultado: '{'prediction_suicidio': False, 'prediction_violencia': True}'
